Question 1 : Calculating Circular Orbits

In [19]:
# question 1.1 (conversion factor)

# constants in the CGS system according to the document
PI = 3.14159
G_CGS = 6.67430e-8 # the gravitational constant in CGS system
c_CGS = 2.99792e10 # the speed of light in CGS system
AU_CGS = 1.49598e13 # 1 AU in CGS system
M_CGS = 1.98854e33 # the mass of the sun in CGS system

# ratio between the escape velocities of an object escaping a planet with the mass of the sun and the radius of 1 AU

# NEW velocity to CGS velocity conversion factor, unit of centimeters per second
velocity_conversion_factor_0 = (G_CGS*M_CGS/AU_CGS)**(1/2)/(1*1/1)**(1/2)

# ratio between the periods of an object orbiting a planet with the mass of the sun and the radius of 1 AU

# NEW time to CGS time conversion factor, unit of seconds
time_conversion_factor_0 = (1/velocity_conversion_factor_0)*(AU_CGS/1)

# rescale to units required for the question
velocity_conversion_factor_1 = velocity_conversion_factor_0*10**(-5) # units to kilometers per second
time_conversion_factor_1 = time_conversion_factor_0 # units not changed (seconds)

print("velocity conversion factor in kilometers per second = ", "{:.11E}".format(velocity_conversion_factor_1), "km/s")
print("time conversion factor in seconds = ", "{:.11E}".format(time_conversion_factor_1), "s")

velocity conversion factor in kilometers per second =  2.97856535548E+01 km/s
time conversion factor in seconds =  5.02248506062E+06 s


In [1]:
# question 1.2 (solving a circular orbit, 20,000 steps)

# constants in the CGS system according to the document
PI = 3.14159
G_CGS = 6.67430e-8 # the gravitational constant in CGS system
c_CGS = 2.99792e10 # the speed of light in CGS system
AU_CGS = 1.49598e13 # 1 AU in CGS system
M_CGS = 1.98854e33 # the mass of the sun in CGS system

# constants in the NEW system according to the document 
G = 1

# fixed variables 
m1, m2, a, e = 1, 1, 2, 0 # 'a' is semi-major axis and 'e' is eccentricity 
N_period, N_step = 100, 20000

# using the method of reduced mass, we can always derive the period of each body in a two-body system (given masses and semi-major axes)
P = 2*PI*(G*(m1+m2))**(-1/2)*a**(3/2)
# in this problem, as the semi-major axis 'a' is identical for both bodies, this value is identical for both bodies
dt = N_period*P/N_step

# also, using the vis-visa equation, we can always derive the instantaneous orbital speed of each body in a two-body system (given masses and semi-major axes)
v_0 = (G*(m1+m2)*(2/a-1/a))**(1/2)/2
# here, the first 'a' is distance between the bodies, and the second 'a' is the semi-major axis of the elliptic path that a body takes
# in this problem, as the system is symmetric, hence the divison of 2

# initial condition (idx = 0), position and velocity

(ri0_x, ri0_y) = (-a/2, 0) 
(vi0_x, vi0_y) = (0, v_0) 
(rj0_x, rj0_y) = (a/2, 0) 
(vj0_x, vj0_y) = (0, -v_0)

# here, we leverage the fact that the bodies are intitially at their apocenters and set the two bodies to start their motion at the x-axis

# initial condition (idx = 0), force and first derivative

r = ((ri0_x-rj0_x)**2+(ri0_y-rj0_y)**2)**(1/2)

ai0_x = -(ri0_x-rj0_x)/r**3 # intial force
ai0_y = -(ri0_y-rj0_y)/r**3
dai0_x = -((vi0_x-vj0_x)/r**3-3*((ri0_x-rj0_x)*(vi0_x-vj0_x)+(ri0_y-rj0_y)*(vi0_y-vj0_y))*(ri0_x-rj0_x)/r**5) # intial first derivative
dai0_y = -((vi0_y-vj0_y)/r**3-3*((ri0_x-rj0_x)*(vi0_x-vj0_x)+(ri0_y-rj0_y)*(vi0_y-vj0_y))*(ri0_y-rj0_y)/r**5)

aj0_x = -(rj0_x-ri0_x)/r**3
aj0_y = -(rj0_y-ri0_y)/r**3
daj0_x = -((vj0_x-vi0_x)/r**3-3*((rj0_x-ri0_x)*(vj0_x-vi0_x)+(rj0_y-ri0_y)*(vj0_y-vi0_y))*(rj0_x-ri0_x)/r**5)
daj0_y = -((vj0_y-vi0_y)/r**3-3*((rj0_x-ri0_x)*(vj0_x-vi0_x)+(rj0_y-ri0_y)*(vj0_y-vi0_y))*(rj0_y-ri0_y)/r**5)

# predictor step (idx = p), position and velocity

rip_x = ri0_x + vi0_x*dt + ai0_x*dt**2/2 + dai0_x*dt**3/6 # predicted position
rip_y = ri0_y + vi0_y*dt + ai0_y*dt**2/2 + dai0_y*dt**3/6

vip_x = vi0_x + ai0_x*dt + dai0_x*dt**2/2 # predicted velocity
vip_y = vi0_y + ai0_y*dt + dai0_y*dt**2/2

rjp_x = rj0_x + vj0_x*dt + aj0_x*dt**2/2 + daj0_x*dt**3/6
rjp_y = rj0_y + vj0_y*dt + aj0_y*dt**2/2 + daj0_y*dt**3/6 

vjp_x = vj0_x + aj0_x*dt + daj0_x*dt**2/2
vjp_y = vj0_y + aj0_y*dt + daj0_y*dt**2/2

# predictor step (idx = p), force and first derivative

aip_x = -(rip_x-rjp_x)/r**3 # prediced force
aip_y = -(rip_y-rjp_y)/r**3 
daip_x = -((vip_x-vjp_x)/r**3-3*((rip_x-rjp_x)*(vip_x-vjp_x)+(rip_y-rjp_y)*(vip_y-vjp_y))*(rip_x-rjp_x)/r**5) # predicted acceleration
daip_y = -((vip_y-vjp_y)/r**3-3*((rip_x-rjp_x)*(vip_x-vjp_x)+(rip_y-rjp_y)*(vip_y-vjp_y))*(rip_y-rjp_y)/r**5) 

ajp_x = -(rjp_x-rip_x)/r**3
ajp_y = -(rjp_y-rip_y)/r**3
dajp_x = -((vjp_x-vip_x)/r**3-3*((rjp_x-rip_x)*(vjp_x-vip_x)+(rjp_y-rip_y)*(vjp_y-vip_y))*(rjp_x-rip_x)/r**5) # predicted first derivative
dajp_y = -((vjp_y-vip_y)/r**3-3*((rjp_x-rip_x)*(vjp_x-vip_x)+(rjp_y-rip_y)*(vjp_y-vip_y))*(rjp_y-rip_y)/r**5) 

# the corrector step (idx = 1), second, and fourth derivative

ddai0_x = -6*(ai0_x-aip_x)/dt**2-2*(2*dai0_x+daip_x)/dt # intial second derivative
ddai0_y = -6*(ai0_y-aip_y)/dt**2-2*(2*dai0_y+daip_y)/dt 
dddai0_x = 12*(ai0_x-aip_x)/dt**3+6*(dai0_x+daip_x)/dt**2 # inital third derivative
dddai0_y = 12*(ai0_y-aip_y)/dt**3+6*(dai0_y+daip_y)/dt**2

ddaj0_x = -6*(aj0_x-ajp_x)/dt**2-2*(2*daj0_x+dajp_x)/dt 
ddaj0_y = -6*(aj0_y-ajp_y)/dt**2-2*(2*daj0_y+dajp_y)/dt
dddaj0_x = 12*(aj0_x-ajp_x)/dt**3+6*(daj0_x+dajp_x)/dt**2 
dddaj0_y = 12*(aj0_y-ajp_y)/dt**3+6*(daj0_y+dajp_y)/dt**2 

ri1_x = rip_x + ddai0_x*dt**4/24 + dddai0_x*dt**5/120 # corrected position
ri1_y = rip_y + ddai0_y*dt**4/24 + dddai0_y*dt**5/120 
vi1_x = vip_x + ddai0_x*dt**3/6 + dddai0_x*dt**4/24 # corrected velocity
vi1_y = vip_y + ddai0_y*dt**3/6 + dddai0_y*dt**4/24

rj1_x = rjp_x + ddaj0_x*dt**4/24 + dddaj0_x*dt**5/120
rj1_y = rjp_y + ddaj0_y*dt**4/24 + dddaj0_y*dt**5/120
vj1_x = vjp_x + ddaj0_x*dt**3/6 + dddaj0_x*dt**4/24
vj1_y = vjp_y + ddaj0_y*dt**3/6 + dddaj0_y*dt**4/24

# for listing
ri_x_list = []
ri_y_list = []
rj_x_list = []
rj_y_list = []

# loop
ri_x_list.append(ri0_x)
ri_y_list.append(ri0_y)
rj_x_list.append(rj0_x)
rj_y_list.append(rj0_y)

for N in range(N_step):
    # initial condition (idx = n-1), force and first derivative

    r = ((ri0_x-rj0_x)**2+(ri0_y-rj0_y)**2)**(1/2)

    ai0_x = -(ri0_x-rj0_x)/r**3 # intial force
    ai0_y = -(ri0_y-rj0_y)/r**3
    dai0_x = -((vi0_x-vj0_x)/r**3-3*((ri0_x-rj0_x)*(vi0_x-vj0_x)+(ri0_y-rj0_y)*(vi0_y-vj0_y))*(ri0_x-rj0_x)/r**5) # intial first derivative
    dai0_y = -((vi0_y-vj0_y)/r**3-3*((ri0_x-rj0_x)*(vi0_x-vj0_x)+(ri0_y-rj0_y)*(vi0_y-vj0_y))*(ri0_y-rj0_y)/r**5)

    aj0_x = -(rj0_x-ri0_x)/r**3
    aj0_y = -(rj0_y-ri0_y)/r**3
    daj0_x = -((vj0_x-vi0_x)/r**3-3*((rj0_x-ri0_x)*(vj0_x-vi0_x)+(rj0_y-ri0_y)*(vj0_y-vi0_y))*(rj0_x-ri0_x)/r**5)
    daj0_y = -((vj0_y-vi0_y)/r**3-3*((rj0_x-ri0_x)*(vj0_x-vi0_x)+(rj0_y-ri0_y)*(vj0_y-vi0_y))*(rj0_y-ri0_y)/r**5)

    # predictor step (idx = p), position and velocity

    rip_x = ri0_x + vi0_x*dt + ai0_x*dt**2/2 + dai0_x*dt**3/6 # predicted position
    rip_y = ri0_y + vi0_y*dt + ai0_y*dt**2/2 + dai0_y*dt**3/6

    vip_x = vi0_x + ai0_x*dt + dai0_x*dt**2/2 # predicted velocity
    vip_y = vi0_y + ai0_y*dt + dai0_y*dt**2/2

    rjp_x = rj0_x + vj0_x*dt + aj0_x*dt**2/2 + daj0_x*dt**3/6
    rjp_y = rj0_y + vj0_y*dt + aj0_y*dt**2/2 + daj0_y*dt**3/6 

    vjp_x = vj0_x + aj0_x*dt + daj0_x*dt**2/2
    vjp_y = vj0_y + aj0_y*dt + daj0_y*dt**2/2

    # predictor step (idx = p), force and first derivative

    aip_x = -(rip_x-rjp_x)/r**3 # prediced force
    aip_y = -(rip_y-rjp_y)/r**3 
    daip_x = -((vip_x-vjp_x)/r**3-3*((rip_x-rjp_x)*(vip_x-vjp_x)+(rip_y-rjp_y)*(vip_y-vjp_y))*(rip_x-rjp_x)/r**5) # predicted velocity
    daip_y = -((vip_y-vjp_y)/r**3-3*((rip_x-rjp_x)*(vip_x-vjp_x)+(rip_y-rjp_y)*(vip_y-vjp_y))*(rip_y-rjp_y)/r**5) 

    ajp_x = -(rjp_x-rip_x)/r**3
    ajp_y = -(rjp_y-rip_y)/r**3
    dajp_x = -((vjp_x-vip_x)/r**3-3*((rjp_x-rip_x)*(vjp_x-vip_x)+(rjp_y-rip_y)*(vjp_y-vip_y))*(rjp_x-rip_x)/r**5) # predicted velocity
    dajp_y = -((vjp_y-vip_y)/r**3-3*((rjp_x-rip_x)*(vjp_x-vip_x)+(rjp_y-rip_y)*(vjp_y-vip_y))*(rjp_y-rip_y)/r**5) 

    # the corrector step (idx = n), second, and fourth derivative

    ddai0_x = -6*(ai0_x-aip_x)/dt**2-2*(2*dai0_x+daip_x)/dt # intial second derivative
    ddai0_y = -6*(ai0_y-aip_y)/dt**2-2*(2*dai0_y+daip_y)/dt 
    dddai0_x = 12*(ai0_x-aip_x)/dt**3+6*(dai0_x+daip_x)/dt**2 # inital third derivative
    dddai0_y = 12*(ai0_y-aip_y)/dt**3+6*(dai0_y+daip_y)/dt**2

    ddaj0_x = -6*(aj0_x-ajp_x)/dt**2-2*(2*daj0_x+dajp_x)/dt 
    ddaj0_y = -6*(aj0_y-ajp_y)/dt**2-2*(2*daj0_y+dajp_y)/dt
    dddaj0_x = 12*(aj0_x-ajp_x)/dt**3+6*(daj0_x+dajp_x)/dt**2 
    dddaj0_y = 12*(aj0_y-ajp_y)/dt**3+6*(daj0_y+dajp_y)/dt**2 

    ri1_x = rip_x + ddai0_x*dt**4/24 + dddai0_x*dt**5/120 # corrected position
    ri1_y = rip_y + ddai0_y*dt**4/24 + dddai0_y*dt**5/120 
    vi1_x = vip_x + ddai0_x*dt**3/6 + dddai0_x*dt**4/24 # corrected velocity
    vi1_y = vip_y + ddai0_y*dt**3/6 + dddai0_y*dt**4/24

    rj1_x = rjp_x + ddaj0_x*dt**4/24 + dddaj0_x*dt**5/120
    rj1_y = rjp_y + ddaj0_y*dt**4/24 + dddaj0_y*dt**5/120
    vj1_x = vjp_x + ddaj0_x*dt**3/6 + dddaj0_x*dt**4/24
    vj1_y = vjp_y + ddaj0_y*dt**3/6 + dddaj0_y*dt**4/24

    # saving information in a list
    ri_x_list.append(ri1_x) 
    ri_y_list.append(ri1_y) 
    rj_x_list.append(rj1_x) 
    rj_y_list.append(rj1_y) 

    # updating quantites
    (ri0_x, ri0_y) = (ri1_x, ri1_y) 
    (vi0_x, vi0_y) = (vi1_x, vi1_y) 
    (rj0_x, rj0_y) = (rj1_x, rj1_y) 
    (vj0_x, vj0_y) = (vj1_x, vj1_y) 

    # print(ri0_x, ri0_y, rj0_x, rj0_y)
    # print(vi0_x, vi0_y, vj0_x, vj0_y)
    # print(vi0_x**2+vi0_y**2)

# plotting the results (optional)
# import pylab 
# pylab.plot(ri_x_list,ri_y_list)
# pylab.plot(rj_x_list,rj_y_list)
# pylab.axis("equal")
# pylab.xlabel("x (AU)")
# pylab.ylabel("y (AU)")
# pylab.show()

print("current step = ", int(N_step), "steps")
print("current time = ", "{:.11e}".format(N_period*P), "unit time")
print("current distance between stars = ", "{:.11e}".format(((ri0_x-rj0_x)**2+(ri0_y-rj0_y)**2)**(1/2)), "unit distance")

current step =  20000 steps
current time =  1.25663600000e+03 unit time
current distance between stars =  2.00000095119e+00 unit distance


In [2]:
# question 1.3 (solving a circular orbit, 40,000 steps)

# constants in the CGS system according to the document
PI = 3.14159
G_CGS = 6.67430e-8 # the gravitational constant in CGS system
c_CGS = 2.99792e10 # the speed of light in CGS system
AU_CGS = 1.49598e13 # 1 AU in CGS system
M_CGS = 1.98854e33 # the mass of the sun in CGS system

# constants in the NEW system according to the document 
G = 1

# fixed variables 
m1, m2, a, e = 1, 1, 2, 0 # 'a' is semi-major axis and 'e' is eccentricity 
N_period, N_step = 100, 40000

# using the method of reduced mass, we can always derive the period of each body in a two-body system (given masses and semi-major axes)
P = 2*PI*(G*(m1+m2))**(-1/2)*a**(3/2)
# in this problem, as the semi-major axis 'a' is identical, this value is identical for both bodies
dt = N_period*P/N_step

# also, using the vis-visa equation, we can always derive the instantaneous orbital speed of each body in a two-body system (given masses and semi-major axes)
v_0 = (G*(m1+m2)*(2/a-1/a))**(1/2)/2
# here, the first 'a' is distance between the bodies, and the second 'a' is the semi-major axis of the elliptic path that a body takes
# in this problem, as the system is symmetric, hence the divison of 2

# initial condition (idx = 0), position and velocity

(ri0_x, ri0_y) = (-a/2, 0) 
(vi0_x, vi0_y) = (0, v_0) 
(rj0_x, rj0_y) = (a/2, 0) 
(vj0_x, vj0_y) = (0, -v_0)

# here, we leverage the fact that the bodies are intitially at their apocenters and set the two bodies to start their motion at the x-axis

# initial condition (idx = 0), force and first derivative

r = ((ri0_x-rj0_x)**2+(ri0_y-rj0_y)**2)**(1/2)

ai0_x = -(ri0_x-rj0_x)/r**3 # intial force
ai0_y = -(ri0_y-rj0_y)/r**3
dai0_x = -((vi0_x-vj0_x)/r**3-3*((ri0_x-rj0_x)*(vi0_x-vj0_x)+(ri0_y-rj0_y)*(vi0_y-vj0_y))*(ri0_x-rj0_x)/r**5) # intial first derivative
dai0_y = -((vi0_y-vj0_y)/r**3-3*((ri0_x-rj0_x)*(vi0_x-vj0_x)+(ri0_y-rj0_y)*(vi0_y-vj0_y))*(ri0_y-rj0_y)/r**5)

aj0_x = -(rj0_x-ri0_x)/r**3
aj0_y = -(rj0_y-ri0_y)/r**3
daj0_x = -((vj0_x-vi0_x)/r**3-3*((rj0_x-ri0_x)*(vj0_x-vi0_x)+(rj0_y-ri0_y)*(vj0_y-vi0_y))*(rj0_x-ri0_x)/r**5)
daj0_y = -((vj0_y-vi0_y)/r**3-3*((rj0_x-ri0_x)*(vj0_x-vi0_x)+(rj0_y-ri0_y)*(vj0_y-vi0_y))*(rj0_y-ri0_y)/r**5)

# predictor step (idx = p), position and velocity

rip_x = ri0_x + vi0_x*dt + ai0_x*dt**2/2 + dai0_x*dt**3/6 # predicted position
rip_y = ri0_y + vi0_y*dt + ai0_y*dt**2/2 + dai0_y*dt**3/6

vip_x = vi0_x + ai0_x*dt + dai0_x*dt**2/2 # predicted velocity
vip_y = vi0_y + ai0_y*dt + dai0_y*dt**2/2

rjp_x = rj0_x + vj0_x*dt + aj0_x*dt**2/2 + daj0_x*dt**3/6
rjp_y = rj0_y + vj0_y*dt + aj0_y*dt**2/2 + daj0_y*dt**3/6 

vjp_x = vj0_x + aj0_x*dt + daj0_x*dt**2/2
vjp_y = vj0_y + aj0_y*dt + daj0_y*dt**2/2

# predictor step (idx = p), force and first derivative

aip_x = -(rip_x-rjp_x)/r**3 # prediced force
aip_y = -(rip_y-rjp_y)/r**3 
daip_x = -((vip_x-vjp_x)/r**3-3*((rip_x-rjp_x)*(vip_x-vjp_x)+(rip_y-rjp_y)*(vip_y-vjp_y))*(rip_x-rjp_x)/r**5) # predicted acceleration
daip_y = -((vip_y-vjp_y)/r**3-3*((rip_x-rjp_x)*(vip_x-vjp_x)+(rip_y-rjp_y)*(vip_y-vjp_y))*(rip_y-rjp_y)/r**5) 

ajp_x = -(rjp_x-rip_x)/r**3
ajp_y = -(rjp_y-rip_y)/r**3
dajp_x = -((vjp_x-vip_x)/r**3-3*((rjp_x-rip_x)*(vjp_x-vip_x)+(rjp_y-rip_y)*(vjp_y-vip_y))*(rjp_x-rip_x)/r**5) # predicted first derivative
dajp_y = -((vjp_y-vip_y)/r**3-3*((rjp_x-rip_x)*(vjp_x-vip_x)+(rjp_y-rip_y)*(vjp_y-vip_y))*(rjp_y-rip_y)/r**5) 

# the corrector step (idx = 1), second, and fourth derivative

ddai0_x = -6*(ai0_x-aip_x)/dt**2-2*(2*dai0_x+daip_x)/dt # intial second derivative
ddai0_y = -6*(ai0_y-aip_y)/dt**2-2*(2*dai0_y+daip_y)/dt 
dddai0_x = 12*(ai0_x-aip_x)/dt**3+6*(dai0_x+daip_x)/dt**2 # inital third derivative
dddai0_y = 12*(ai0_y-aip_y)/dt**3+6*(dai0_y+daip_y)/dt**2

ddaj0_x = -6*(aj0_x-ajp_x)/dt**2-2*(2*daj0_x+dajp_x)/dt 
ddaj0_y = -6*(aj0_y-ajp_y)/dt**2-2*(2*daj0_y+dajp_y)/dt
dddaj0_x = 12*(aj0_x-ajp_x)/dt**3+6*(daj0_x+dajp_x)/dt**2 
dddaj0_y = 12*(aj0_y-ajp_y)/dt**3+6*(daj0_y+dajp_y)/dt**2 

ri1_x = rip_x + ddai0_x*dt**4/24 + dddai0_x*dt**5/120 # corrected position
ri1_y = rip_y + ddai0_y*dt**4/24 + dddai0_y*dt**5/120 
vi1_x = vip_x + ddai0_x*dt**3/6 + dddai0_x*dt**4/24 # corrected velocity
vi1_y = vip_y + ddai0_y*dt**3/6 + dddai0_y*dt**4/24

rj1_x = rjp_x + ddaj0_x*dt**4/24 + dddaj0_x*dt**5/120
rj1_y = rjp_y + ddaj0_y*dt**4/24 + dddaj0_y*dt**5/120
vj1_x = vjp_x + ddaj0_x*dt**3/6 + dddaj0_x*dt**4/24
vj1_y = vjp_y + ddaj0_y*dt**3/6 + dddaj0_y*dt**4/24

# for listing
ri_x_list = []
ri_y_list = []
rj_x_list = []
rj_y_list = []

# loop
ri_x_list.append(ri0_x)
ri_y_list.append(ri0_y)
rj_x_list.append(rj0_x)
rj_y_list.append(rj0_y)

for N in range(N_step):
    # initial condition (idx = n-1), force and first derivative

    r = ((ri0_x-rj0_x)**2+(ri0_y-rj0_y)**2)**(1/2)

    ai0_x = -(ri0_x-rj0_x)/r**3 # intial force
    ai0_y = -(ri0_y-rj0_y)/r**3
    dai0_x = -((vi0_x-vj0_x)/r**3-3*((ri0_x-rj0_x)*(vi0_x-vj0_x)+(ri0_y-rj0_y)*(vi0_y-vj0_y))*(ri0_x-rj0_x)/r**5) # intial first derivative
    dai0_y = -((vi0_y-vj0_y)/r**3-3*((ri0_x-rj0_x)*(vi0_x-vj0_x)+(ri0_y-rj0_y)*(vi0_y-vj0_y))*(ri0_y-rj0_y)/r**5)

    aj0_x = -(rj0_x-ri0_x)/r**3
    aj0_y = -(rj0_y-ri0_y)/r**3
    daj0_x = -((vj0_x-vi0_x)/r**3-3*((rj0_x-ri0_x)*(vj0_x-vi0_x)+(rj0_y-ri0_y)*(vj0_y-vi0_y))*(rj0_x-ri0_x)/r**5)
    daj0_y = -((vj0_y-vi0_y)/r**3-3*((rj0_x-ri0_x)*(vj0_x-vi0_x)+(rj0_y-ri0_y)*(vj0_y-vi0_y))*(rj0_y-ri0_y)/r**5)

    # predictor step (idx = p), position and velocity

    rip_x = ri0_x + vi0_x*dt + ai0_x*dt**2/2 + dai0_x*dt**3/6 # predicted position
    rip_y = ri0_y + vi0_y*dt + ai0_y*dt**2/2 + dai0_y*dt**3/6

    vip_x = vi0_x + ai0_x*dt + dai0_x*dt**2/2 # predicted velocity
    vip_y = vi0_y + ai0_y*dt + dai0_y*dt**2/2

    rjp_x = rj0_x + vj0_x*dt + aj0_x*dt**2/2 + daj0_x*dt**3/6
    rjp_y = rj0_y + vj0_y*dt + aj0_y*dt**2/2 + daj0_y*dt**3/6 

    vjp_x = vj0_x + aj0_x*dt + daj0_x*dt**2/2
    vjp_y = vj0_y + aj0_y*dt + daj0_y*dt**2/2

    # predictor step (idx = p), force and first derivative

    aip_x = -(rip_x-rjp_x)/r**3 # prediced force
    aip_y = -(rip_y-rjp_y)/r**3 
    daip_x = -((vip_x-vjp_x)/r**3-3*((rip_x-rjp_x)*(vip_x-vjp_x)+(rip_y-rjp_y)*(vip_y-vjp_y))*(rip_x-rjp_x)/r**5) # predicted velocity
    daip_y = -((vip_y-vjp_y)/r**3-3*((rip_x-rjp_x)*(vip_x-vjp_x)+(rip_y-rjp_y)*(vip_y-vjp_y))*(rip_y-rjp_y)/r**5) 

    ajp_x = -(rjp_x-rip_x)/r**3
    ajp_y = -(rjp_y-rip_y)/r**3
    dajp_x = -((vjp_x-vip_x)/r**3-3*((rjp_x-rip_x)*(vjp_x-vip_x)+(rjp_y-rip_y)*(vjp_y-vip_y))*(rjp_x-rip_x)/r**5) # predicted velocity
    dajp_y = -((vjp_y-vip_y)/r**3-3*((rjp_x-rip_x)*(vjp_x-vip_x)+(rjp_y-rip_y)*(vjp_y-vip_y))*(rjp_y-rip_y)/r**5) 

    # the corrector step (idx = n), second, and fourth derivative

    ddai0_x = -6*(ai0_x-aip_x)/dt**2-2*(2*dai0_x+daip_x)/dt # intial second derivative
    ddai0_y = -6*(ai0_y-aip_y)/dt**2-2*(2*dai0_y+daip_y)/dt 
    dddai0_x = 12*(ai0_x-aip_x)/dt**3+6*(dai0_x+daip_x)/dt**2 # inital third derivative
    dddai0_y = 12*(ai0_y-aip_y)/dt**3+6*(dai0_y+daip_y)/dt**2

    ddaj0_x = -6*(aj0_x-ajp_x)/dt**2-2*(2*daj0_x+dajp_x)/dt 
    ddaj0_y = -6*(aj0_y-ajp_y)/dt**2-2*(2*daj0_y+dajp_y)/dt
    dddaj0_x = 12*(aj0_x-ajp_x)/dt**3+6*(daj0_x+dajp_x)/dt**2 
    dddaj0_y = 12*(aj0_y-ajp_y)/dt**3+6*(daj0_y+dajp_y)/dt**2 

    ri1_x = rip_x + ddai0_x*dt**4/24 + dddai0_x*dt**5/120 # corrected position
    ri1_y = rip_y + ddai0_y*dt**4/24 + dddai0_y*dt**5/120 
    vi1_x = vip_x + ddai0_x*dt**3/6 + dddai0_x*dt**4/24 # corrected velocity
    vi1_y = vip_y + ddai0_y*dt**3/6 + dddai0_y*dt**4/24

    rj1_x = rjp_x + ddaj0_x*dt**4/24 + dddaj0_x*dt**5/120
    rj1_y = rjp_y + ddaj0_y*dt**4/24 + dddaj0_y*dt**5/120
    vj1_x = vjp_x + ddaj0_x*dt**3/6 + dddaj0_x*dt**4/24
    vj1_y = vjp_y + ddaj0_y*dt**3/6 + dddaj0_y*dt**4/24

    # saving information in a list
    ri_x_list.append(ri1_x) 
    ri_y_list.append(ri1_y) 
    rj_x_list.append(rj1_x) 
    rj_y_list.append(rj1_y) 

    # updating quantites
    (ri0_x, ri0_y) = (ri1_x, ri1_y) 
    (vi0_x, vi0_y) = (vi1_x, vi1_y) 
    (rj0_x, rj0_y) = (rj1_x, rj1_y) 
    (vj0_x, vj0_y) = (vj1_x, vj1_y) 

    # print(ri0_x, ri0_y, rj0_x, rj0_y)
    # print(vi0_x, vi0_y, vj0_x, vj0_y)
    # print(vi0_x**2+vi0_y**2)

# plotting the results (optional)
# import pylab 
# pylab.plot(ri_x_list,ri_y_list)
# pylab.plot(rj_x_list,rj_y_list)
# pylab.axis("equal")
# pylab.xlabel("x (AU)")
# pylab.ylabel("y (AU)")
# pylab.show()

print("current step = ", int(N_step), "steps")
print("current time = ", "{:.11e}".format(N_period*P), "unit time")
print("current distance between stars = ", "{:.11e}".format(((ri0_x-rj0_x)**2+(ri0_y-rj0_y)**2)**(1/2)), "unit distance")

current step =  40000 steps
current time =  1.25663600000e+03 unit time
current distance between stars =  2.00000003278e+00 unit distance


In [3]:
# question 1.4 (solving a circular orbit, black hole & neutron star system, 20,000 steps)

# constants in the CGS system according to the document
PI = 3.14159
G_CGS = 6.67430e-8 # the gravitational constant in CGS system
c_CGS = 2.99792e10 # the speed of light in CGS system
AU_CGS = 1.49598e13 # 1 AU in CGS system
M_CGS = 1.98854e33 # the mass of the sun in CGS system

# constants in the NEW system according to the document 
G = 1

# fixed variables 
m1, m2, a, e = 10, 2, 100, 0 # 'a' is semi-major axis and 'e' is eccentricity 
N_period, N_step = 10, 20000

# using the method of reduced mass, we can always derive the period of each body in a two-body system (given masses and semi-major axes)
P = 2*PI*(G*(m1+m2))**(-1/2)*a**(3/2)
# in this problem, as the semi-major axis 'a' is identical, this value is identical for both bodies
dt = N_period*P/N_step

# also, using the vis-visa equation, we can always derive the instantaneous orbital speed of each body in a two-body system (given masses and semi-major axes)
v_i = (G*(m1+m2)*(2/a-1/a))**(1/2)*(m2/(m1+m2))
v_j = (G*(m1+m2)*(2/a-1/a))**(1/2)*(-m1/(m1+m2))
# here, the first 'a' is distance between the bodies, and the second 'a' is the semi-major axis of the elliptic path that a body takes
# in this problem, as the system is asymmetric, conservation of linear momentum was used to find each value

# initial condition (idx = 0), position and velocity

(ri0_x, ri0_y) = (-a*m2/(m1+m2), 0) 
(vi0_x, vi0_y) = (0, v_i) 
(rj0_x, rj0_y) = (a*m1/(m1+m2), 0) 
(vj0_x, vj0_y) = (0, v_j)

# here, we leverage the fact that the bodies are intitially at their apocenters and set the two bodies to start their motion at the x-axis

# initial condition (idx = 0), force and first derivative

r = ((ri0_x-rj0_x)**2+(ri0_y-rj0_y)**2)**(1/2)

ai0_x = -m2*(ri0_x-rj0_x)/r**3 # intial force
ai0_y = -m2*(ri0_y-rj0_y)/r**3
dai0_x = -m2*((vi0_x-vj0_x)/r**3-3*((ri0_x-rj0_x)*(vi0_x-vj0_x)+(ri0_y-rj0_y)*(vi0_y-vj0_y))*(ri0_x-rj0_x)/r**5) # intial first derivative
dai0_y = -m2*((vi0_y-vj0_y)/r**3-3*((ri0_x-rj0_x)*(vi0_x-vj0_x)+(ri0_y-rj0_y)*(vi0_y-vj0_y))*(ri0_y-rj0_y)/r**5)

aj0_x = -m1*(rj0_x-ri0_x)/r**3
aj0_y = -m1*(rj0_y-ri0_y)/r**3
daj0_x = -m1*((vj0_x-vi0_x)/r**3-3*((rj0_x-ri0_x)*(vj0_x-vi0_x)+(rj0_y-ri0_y)*(vj0_y-vi0_y))*(rj0_x-ri0_x)/r**5)
daj0_y = -m1*((vj0_y-vi0_y)/r**3-3*((rj0_x-ri0_x)*(vj0_x-vi0_x)+(rj0_y-ri0_y)*(vj0_y-vi0_y))*(rj0_y-ri0_y)/r**5)

# predictor step (idx = p), position and velocity

rip_x = ri0_x + vi0_x*dt + ai0_x*dt**2/2 + dai0_x*dt**3/6 # predicted position
rip_y = ri0_y + vi0_y*dt + ai0_y*dt**2/2 + dai0_y*dt**3/6

vip_x = vi0_x + ai0_x*dt + dai0_x*dt**2/2 # predicted velocity
vip_y = vi0_y + ai0_y*dt + dai0_y*dt**2/2

rjp_x = rj0_x + vj0_x*dt + aj0_x*dt**2/2 + daj0_x*dt**3/6
rjp_y = rj0_y + vj0_y*dt + aj0_y*dt**2/2 + daj0_y*dt**3/6 

vjp_x = vj0_x + aj0_x*dt + daj0_x*dt**2/2
vjp_y = vj0_y + aj0_y*dt + daj0_y*dt**2/2

# predictor step (idx = p), force and first derivative

aip_x = -m2*(rip_x-rjp_x)/r**3 # prediced force
aip_y = -m2*(rip_y-rjp_y)/r**3 
daip_x = -m2*((vip_x-vjp_x)/r**3-3*((rip_x-rjp_x)*(vip_x-vjp_x)+(rip_y-rjp_y)*(vip_y-vjp_y))*(rip_x-rjp_x)/r**5) # predicted acceleration
daip_y = -m2*((vip_y-vjp_y)/r**3-3*((rip_x-rjp_x)*(vip_x-vjp_x)+(rip_y-rjp_y)*(vip_y-vjp_y))*(rip_y-rjp_y)/r**5) 

ajp_x = -m1*(rjp_x-rip_x)/r**3
ajp_y = -m1*(rjp_y-rip_y)/r**3
dajp_x = -m1*((vjp_x-vip_x)/r**3-3*((rjp_x-rip_x)*(vjp_x-vip_x)+(rjp_y-rip_y)*(vjp_y-vip_y))*(rjp_x-rip_x)/r**5) # predicted first derivative
dajp_y = -m1*((vjp_y-vip_y)/r**3-3*((rjp_x-rip_x)*(vjp_x-vip_x)+(rjp_y-rip_y)*(vjp_y-vip_y))*(rjp_y-rip_y)/r**5) 

# the corrector step (idx = 1), second, and fourth derivative

ddai0_x = -6*(ai0_x-aip_x)/dt**2-2*(2*dai0_x+daip_x)/dt # intial second derivative
ddai0_y = -6*(ai0_y-aip_y)/dt**2-2*(2*dai0_y+daip_y)/dt 
dddai0_x = 12*(ai0_x-aip_x)/dt**3+6*(dai0_x+daip_x)/dt**2 # inital third derivative
dddai0_y = 12*(ai0_y-aip_y)/dt**3+6*(dai0_y+daip_y)/dt**2

ddaj0_x = -6*(aj0_x-ajp_x)/dt**2-2*(2*daj0_x+dajp_x)/dt 
ddaj0_y = -6*(aj0_y-ajp_y)/dt**2-2*(2*daj0_y+dajp_y)/dt
dddaj0_x = 12*(aj0_x-ajp_x)/dt**3+6*(daj0_x+dajp_x)/dt**2 
dddaj0_y = 12*(aj0_y-ajp_y)/dt**3+6*(daj0_y+dajp_y)/dt**2 

ri1_x = rip_x + ddai0_x*dt**4/24 + dddai0_x*dt**5/120 # corrected position
ri1_y = rip_y + ddai0_y*dt**4/24 + dddai0_y*dt**5/120 
vi1_x = vip_x + ddai0_x*dt**3/6 + dddai0_x*dt**4/24 # corrected velocity
vi1_y = vip_y + ddai0_y*dt**3/6 + dddai0_y*dt**4/24

rj1_x = rjp_x + ddaj0_x*dt**4/24 + dddaj0_x*dt**5/120
rj1_y = rjp_y + ddaj0_y*dt**4/24 + dddaj0_y*dt**5/120
vj1_x = vjp_x + ddaj0_x*dt**3/6 + dddaj0_x*dt**4/24
vj1_y = vjp_y + ddaj0_y*dt**3/6 + dddaj0_y*dt**4/24

# for listing
ri_x_list = []
ri_y_list = []
rj_x_list = []
rj_y_list = []

# loop
ri_x_list.append(ri0_x)
ri_y_list.append(ri0_y)
rj_x_list.append(rj0_x)
rj_y_list.append(rj0_y)

for N in range(N_step):
    # initial condition (idx = n-1), force and first derivative

    r = ((ri0_x-rj0_x)**2+(ri0_y-rj0_y)**2)**(1/2)

    ai0_x = -m2*(ri0_x-rj0_x)/r**3 # intial force
    ai0_y = -m2*(ri0_y-rj0_y)/r**3
    dai0_x = -m2*((vi0_x-vj0_x)/r**3-3*((ri0_x-rj0_x)*(vi0_x-vj0_x)+(ri0_y-rj0_y)*(vi0_y-vj0_y))*(ri0_x-rj0_x)/r**5) # intial first derivative
    dai0_y = -m2*((vi0_y-vj0_y)/r**3-3*((ri0_x-rj0_x)*(vi0_x-vj0_x)+(ri0_y-rj0_y)*(vi0_y-vj0_y))*(ri0_y-rj0_y)/r**5)

    aj0_x = -m1*(rj0_x-ri0_x)/r**3
    aj0_y = -m1*(rj0_y-ri0_y)/r**3
    daj0_x = -m1*((vj0_x-vi0_x)/r**3-3*((rj0_x-ri0_x)*(vj0_x-vi0_x)+(rj0_y-ri0_y)*(vj0_y-vi0_y))*(rj0_x-ri0_x)/r**5)
    daj0_y = -m1*((vj0_y-vi0_y)/r**3-3*((rj0_x-ri0_x)*(vj0_x-vi0_x)+(rj0_y-ri0_y)*(vj0_y-vi0_y))*(rj0_y-ri0_y)/r**5)

    # predictor step (idx = p), position and velocity

    rip_x = ri0_x + vi0_x*dt + ai0_x*dt**2/2 + dai0_x*dt**3/6 # predicted position
    rip_y = ri0_y + vi0_y*dt + ai0_y*dt**2/2 + dai0_y*dt**3/6

    vip_x = vi0_x + ai0_x*dt + dai0_x*dt**2/2 # predicted velocity
    vip_y = vi0_y + ai0_y*dt + dai0_y*dt**2/2

    rjp_x = rj0_x + vj0_x*dt + aj0_x*dt**2/2 + daj0_x*dt**3/6
    rjp_y = rj0_y + vj0_y*dt + aj0_y*dt**2/2 + daj0_y*dt**3/6 

    vjp_x = vj0_x + aj0_x*dt + daj0_x*dt**2/2
    vjp_y = vj0_y + aj0_y*dt + daj0_y*dt**2/2

    # predictor step (idx = p), force and first derivative

    aip_x = -m2*(rip_x-rjp_x)/r**3 # prediced force
    aip_y = -m2*(rip_y-rjp_y)/r**3 
    daip_x = -m2*((vip_x-vjp_x)/r**3-3*((rip_x-rjp_x)*(vip_x-vjp_x)+(rip_y-rjp_y)*(vip_y-vjp_y))*(rip_x-rjp_x)/r**5) # predicted velocity
    daip_y = -m2*((vip_y-vjp_y)/r**3-3*((rip_x-rjp_x)*(vip_x-vjp_x)+(rip_y-rjp_y)*(vip_y-vjp_y))*(rip_y-rjp_y)/r**5) 

    ajp_x = -m1*(rjp_x-rip_x)/r**3
    ajp_y = -m1*(rjp_y-rip_y)/r**3
    dajp_x = -m1*((vjp_x-vip_x)/r**3-3*((rjp_x-rip_x)*(vjp_x-vip_x)+(rjp_y-rip_y)*(vjp_y-vip_y))*(rjp_x-rip_x)/r**5) # predicted velocity
    dajp_y = -m1*((vjp_y-vip_y)/r**3-3*((rjp_x-rip_x)*(vjp_x-vip_x)+(rjp_y-rip_y)*(vjp_y-vip_y))*(rjp_y-rip_y)/r**5) 

    # the corrector step (idx = n), second, and fourth derivative

    ddai0_x = -6*(ai0_x-aip_x)/dt**2-2*(2*dai0_x+daip_x)/dt # intial second derivative
    ddai0_y = -6*(ai0_y-aip_y)/dt**2-2*(2*dai0_y+daip_y)/dt 
    dddai0_x = 12*(ai0_x-aip_x)/dt**3+6*(dai0_x+daip_x)/dt**2 # inital third derivative
    dddai0_y = 12*(ai0_y-aip_y)/dt**3+6*(dai0_y+daip_y)/dt**2

    ddaj0_x = -6*(aj0_x-ajp_x)/dt**2-2*(2*daj0_x+dajp_x)/dt 
    ddaj0_y = -6*(aj0_y-ajp_y)/dt**2-2*(2*daj0_y+dajp_y)/dt
    dddaj0_x = 12*(aj0_x-ajp_x)/dt**3+6*(daj0_x+dajp_x)/dt**2 
    dddaj0_y = 12*(aj0_y-ajp_y)/dt**3+6*(daj0_y+dajp_y)/dt**2 

    ri1_x = rip_x + ddai0_x*dt**4/24 + dddai0_x*dt**5/120 # corrected position
    ri1_y = rip_y + ddai0_y*dt**4/24 + dddai0_y*dt**5/120 
    vi1_x = vip_x + ddai0_x*dt**3/6 + dddai0_x*dt**4/24 # corrected velocity
    vi1_y = vip_y + ddai0_y*dt**3/6 + dddai0_y*dt**4/24

    rj1_x = rjp_x + ddaj0_x*dt**4/24 + dddaj0_x*dt**5/120
    rj1_y = rjp_y + ddaj0_y*dt**4/24 + dddaj0_y*dt**5/120
    vj1_x = vjp_x + ddaj0_x*dt**3/6 + dddaj0_x*dt**4/24
    vj1_y = vjp_y + ddaj0_y*dt**3/6 + dddaj0_y*dt**4/24

    # saving information in a list
    ri_x_list.append(ri1_x) 
    ri_y_list.append(ri1_y) 
    rj_x_list.append(rj1_x) 
    rj_y_list.append(rj1_y) 

    # updating quantites
    (ri0_x, ri0_y) = (ri1_x, ri1_y) 
    (vi0_x, vi0_y) = (vi1_x, vi1_y) 
    (rj0_x, rj0_y) = (rj1_x, rj1_y) 
    (vj0_x, vj0_y) = (vj1_x, vj1_y) 

    # print(ri0_x, ri0_y, rj0_x, rj0_y)
    # print(vi0_x, vi0_y, vj0_x, vj0_y)
    # print(vi0_x**2+vi0_y**2)

# plotting the results (optional)
# import pylab 
# pylab.plot(ri_x_list,ri_y_list)
# pylab.plot(rj_x_list,rj_y_list)
# pylab.axis("equal")
# pylab.xlabel("x (AU)")
# pylab.ylabel("y (AU)")
# pylab.show()

print("current step = ", int(N_step), "steps")
print("current time = ", "{:.11e}".format(N_period*P), "unit time")
print("current distance between stars = ", "{:.11e}".format(((ri0_x-rj0_x)**2+(ri0_y-rj0_y)**2)**(1/2)), "unit distance")

current step =  20000 steps
current time =  1.81379783218e+04 unit time
current distance between stars =  1.00000000000e+02 unit distance
